In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calculate_force(positions, verlet_list, box_length, k):
    forces = np.zeros_like(positions)
    for i in range(len(positions)):
        for j in verlet_list[i]:
            rij = positions[j] - positions[i]
            rij = rij - np.round(rij / box_length) * box_length  # PBC
            if np.linalg.norm(rij)<=0.8:
                forces[i] += -k *rij* (0.8/np.linalg.norm(rij)-1)
    return forces

In [3]:
def verlet_list(positions, box_length, cutoff):
    verlet_list = []

    for i in range(len(positions)):
        verlet_list.append([])
        for j in range(len(positions)):
            if i != j:
                rij = positions[j] - positions[i]
                rij = rij - np.round(rij / box_length) * box_length  # PBC
                r = np.linalg.norm(rij)
                if r < cutoff:
                    verlet_list[i].append(j)

    return verlet_list

In [4]:
def velocity_verlet(positions,theta, velocities,omiga, forces, dt, mass,jm, box_length, k, verlet_list,fran,frano,gamma,gammao,f0):
    fact=np.zeros_like(forces)
    for i in range(num_particles):
        fact[0]=f0*np.cos(theta[i])
        fact[1]=f0*np.sin(theta[i])
    fmo=-gamma*velocities
    fmoo=-gammao*omiga
    forcestot=forces+fmo+fran+fact
    accel = forcestot / mass
    accelomiga=(fmoo+frano)/jm
    # 更新位置
    positions += velocities * dt + 0.5 * accel * dt**2
    positions = positions % box_length  # PBC
    theta+=omiga*dt+0.5*accelomiga*dt**2
    # 计算新的受力
    new_forces = calculate_force(positions, verlet_list, box_length, k)

    # 更新速度
    velocities += 0.5 * (forces + new_forces+2*(fmo+fran+fact)) / mass * dt
    omiga +=(fmoo+frano)/jm*dt
    # 更新受力
    forces = new_forces

    return positions, velocities, forces

In [5]:
# 模拟参数
num_particles = 90
num_steps = 16000
dt = 0.002
mass = 0.015
d=0.8
jm=mass*d*d/10
box_length = 15.0
k = 6
cutoff = 4
verlet_cutoff = 5
gamma=0.005
gammao=gamma*d*d/3
f0=28
# 初始化位置和速度
positions = np.random.rand(num_particles, 2) * box_length
frans=np.random.randn(num_steps,num_particles, 2)*0.8
franos=np.random.randn(num_steps,num_particles)*0.8*d*d*6
velocities = np.random.randn(num_particles, 2)
forces = np.zeros_like(positions)
theta=np.random.rand(num_particles)*2*np.pi
omiga=np.random.rand(num_particles)
v = np.random.randn(num_particles)
# Verlet 列表
verlet_listnew = verlet_list(positions, box_length, verlet_cutoff)

# 保存历史信息
history = np.zeros((num_steps, num_particles, 2))
historyo= np.zeros((num_steps, num_particles))
history[0] = positions
historyo[0] = omiga

In [ ]:
for step in range(1, num_steps):
    fran=frans[step]
    frano=franos[step]
    positions, velocities, forces = velocity_verlet(positions,theta, velocities,omiga, forces, dt, mass,jm, box_length, k, verlet_listnew,fran,frano,gamma,gammao,f0)
    if step%5==2:
        verlet_listnew=verlet_list(positions, box_length, verlet_cutoff)
        for ii in range(num_particles):
            v[ii]=np.linalg.norm(velocities[ii])
        maxindex  = np.argmax(v)    
        velocities[maxindex]=velocities[maxindex]/2
    history[step] = positions
    historyo[step]= omiga

np.save('historyunderf28.npy',history)
np.save('historyounderf28.npy',historyo)